In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/genaiproj/AddressCorrector')

# Reading data

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet("data/address_with_instructions.parquet")
df.head()


,OID_,AddNum_Pre,Add_Number,AddNum_Suf,AddNo_Full,St_PreMod,St_PreDir,St_PreTyp,St_PreSep,St_Name,...,SecondaryAddress,CityStateZip,FullAddress,FormattedFullAddress,task1_instruction,task1_groundtruth,task2_instruction,task2_groundtruth,noise_level,variant_idx
0,62241576,,1187.0,,1187.0,,north,,,pownal,...,,"pownal, vt, 5260","1187 north pownal road\n\npownal, vt, 5260","1187, north Pownal Road, Bennington County, Ve...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1187""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1187""...",medium,0
1,19781905,,1124.0,,1124.0,,,,,judah bear,...,,"richmond, ky, 40475","1124 judah bear boulevard\n\nrichmond, ky, 40475","1124, Judah Bear Boulevard, Richmond, Madison ...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1124""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1124""...",extreme,2
2,21790301,,27.0,,27.0,,,,,captain bellamy,...,,"ma, 2632","27 captain bellamy lane\n\nma, 2632","27, Captain Bellamy Lane, Centerville, Barnsta...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...",medium,0
3,29482843,,1676.0,,1676.0,,,,,englewood,...,,"mn, 55104-1113","1676 englewood avenue\n\nmn, 55104-1113","1676, Englewood Avenue, Ramsey County, Minneso...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1676""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1676""...",extreme,2
4,74441520,,3820.0,,3820.0,,,,,duckhorn,...,"apt 2104, apt 2104","sacramento, ca, 95834-1363","3820 duckhorn drive\napt 2104, apt 2104\nsacra...","3820, Duckhorn Drive, Unit apt 2104, Sacrament...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""3820""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""3820""...",medium,0


In [ ]:
# getting only mass data
df=df.loc[df['State']=='ma']
df.head()

,OID_,AddNum_Pre,Add_Number,AddNum_Suf,AddNo_Full,St_PreMod,St_PreDir,St_PreTyp,St_PreSep,St_Name,...,SecondaryAddress,CityStateZip,FullAddress,FormattedFullAddress,task1_instruction,task1_groundtruth,task2_instruction,task2_groundtruth,noise_level,variant_idx
2,21790301,,27.0,,27.0,,,,,captain bellamy,...,,"ma, 2632","27 captain bellamy lane\n\nma, 2632","27, Captain Bellamy Lane, Centerville, Barnsta...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""27"",\...",medium,0
16,24211894,,42.0,,42.0,,,,,vadnais,...,,"ma, 1001","42 vadnais street\n\nma, 1001","42, Vadnais Street, Agawam, Hampden County, Ma...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""42"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""42"",\...",medium,0
97,25195295,,28.0,,28.0,,,,,woolworth,...,,"ma, 1106","28 woolworth street\n\nma, 1106","28, Woolworth Street, Longmeadow, Hampden Coun...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""28"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""28"",\...",medium,0
134,22248751,,75.0,,75.0,,,,,willow brook,...,,"ma, 1520","75 willow brook road\n\nma, 1520","75, Willow Brook Road, Holden, Worcester Count...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""75"",\...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""75"",\...",extreme,2
301,24346545,,1990.0,,1990.0,,,,,columbus,...,"137, 137","ma, 2119","1990 columbus avenue\n137, 137\nma, 2119","1990, Columbus Avenue, Unit 137, Roxbury, Suff...",Parse the following address into a structured ...,"{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1990""...","Fix the formatting, structure, correct any exi...","{\n ""AddNum_Pre"": """",\n ""Add_Number"": ""1990""...",extreme,2


In [ ]:
df.shape

(30000, 71)

In [ ]:
df=df.iloc[:20000]

# SFT

 Step 1: Prepare Multi-Instruction Dataset

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
# Step 1: Create base instruction-output pairs
df_task1 = pd.DataFrame({
    "instruction": df["task1_instruction"],
    "output": "System: " + df["task1_groundtruth"].astype(str)
})

df_task2 = pd.DataFrame({
    "instruction": df["task2_instruction"],
    "output": "System: " + df["task2_groundtruth"].astype(str)
})

combined = pd.concat([df_task1, df_task2], ignore_index=True)

df_sample = combined.sample(n=20_000, random_state=42).reset_index(drop=True)

# Step 2: Convert to HF Dataset
dataset = Dataset.from_pandas(df_sample)

# Step 3: Convert to LLaMA-style messages
def format_llama_chat(example):
    return {
        "messages": [
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["output"]}
        ]
    }

dataset = dataset.map(format_llama_chat)


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

{'instruction': 'Fix the formatting, structure, correct any existing entities, or predict/add new values to the appropriate entities of this Address JSON. Expand common abbreviations (like st→street, ave→avenue), correct obvious errors (like leading zeros in numbers), generate new values to the appropriate entities, and standardize capitalization. Keep empty fields as empty strings. Do not return anything other than corrected Address JSON\nAddress JSON: {\n  "AddNum_Pre": "",\n  "Add_Number": "",\n  "AddNum_Suf": "",\n  "St_PreDir": "",\n  "St_Name": "windemere",\n  "St_PosTyp": "street",\n  "St_PosDir": "",\n  "Building": "",\n  "Floor": "",\n  "Unit": "",\n  "Room": "",\n  "Uninc_Comm": "xpringfeld",\n  "Inc_Muni": "",\n  "County": "",\n  "State": "ma",\n  "Zip_Code": "11049"\n}',
 'output': 'System: {\n  "AddNum_Pre": "",\n  "Add_Number": "60",\n  "AddNum_Suf": "",\n  "St_PreDir": "",\n  "St_Name": "windemere",\n  "St_PosTyp": "street",\n  "St_PosDir": "",\n  "Building": "",\n  "Flo

In [ ]:
print(dataset[3]["messages"][0]["content"])


Parse the following address into a structured JSON with these fields: AddNum_Pre, Add_Number, AddNum_Suf, St_PreDir, St_Name, St_PosTyp, St_PosDir, Building, Floor, Unit, Room, Uninc_Comm, Inc_Muni, County, State, Zip_Code.
Address: 309, D Street, Unit 4, South Boston, Suffolk County, Massachusetts, 02127


In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


# Patch tokenizer with the correct template
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Apply tokenizer formatting to build final text strings
def format_to_text(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False,
        )
    }

# Run it over your full dataset
dataset = dataset.map(format_to_text)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import TrainingArguments
from trl import SFTTrainer

chunk_size = 100000
chunks = [dataset.select(range(i, min(i + chunk_size, len(dataset)))) for i in range(0, len(dataset), chunk_size)]


# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    use_rslora = False,
    use_gradient_checkpointing = "unsloth",
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

# Prepare model for training
model = FastLanguageModel.for_training(model, use_gradient_checkpointing=True)

# Training arguments
args = TrainingArguments(
    output_dir = "models/llama3_sft_sfttrainer_MA",
    logging_dir = "outputs/logs",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 2,
    num_train_epochs = 1,
    logging_steps = 10,
    save_strategy = "epoch",
    save_total_limit = 2,
    learning_rate = 2e-4,
    fp16 = True,
    bf16 = False,
    optim = "adamw_8bit",
    report_to = "none",
    lr_scheduler_type = "linear",
    seed = 3407,
)

# Training loop using SFTTrainer
for i, chunk in enumerate(chunks):
    print(f"\n🚀 Training chunk {i+1}/{len(chunks)}...")

    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = chunk,
        dataset_text_field = "text",
        max_seq_length = 2048,
        packing = False,
        args = args,
    )

    trainer.train()

print("\n All chunks trained with SFTTrainer!")


🚀 Training chunk 1/1...


Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/20000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1 | Total steps = 2,500
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.570700
20,0.440400
30,0.332400
40,0.335900
50,0.293100
60,0.307800
70,0.305900
80,0.251500
90,0.241300
100,0.235700



✅ All chunks trained with SFTTrainer!


Inference

In [ ]:
# same session as training inference - for parsing(task 1)
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template

# Re-apply tokenizer template
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Task 1 Prompt
messages = [
    {
        "role": "user",
        "content": '''Parse the following address into a structured JSON with these fields: AddNum_Pre, Add_Number, AddNum_Suf, St_PreDir, St_Name, St_PosTyp, St_PosDir, Building, Floor, Unit, Room, Uninc_Comm, Inc_Muni, County, State, Zip_Code.
Address: 47 W 13th St, South Boston, MA 02127'''
    }
]

# Format prompt
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

_ = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)


System: {
  "AddNum_Pre": "",
  "Add_Number": "47",
  "AddNum_Suf": "",
  "St_PreDir": "",
  "St_Name": "w 13th",
  "St_PosTyp": "st",
  "St_PosDir": "",
  "Building": "",
  "Floor": "",
  "Unit": "",
  "Room": "",
  "Uninc_Comm": "south boston",
  "Inc_Muni": "boston",
  "County": "suffolk",
  "State": "ma",
  "Zip_Code": "2127"
}


In [ ]:
# same session inference - for entity generation and rewriting(task 2)

from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

# Patch tokenizer for LLaMA 3 chat format
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Define instruction or address JSON
messages = [
    {
        "role": "user",
        "content": '''Fix the formatting, structure, correct any existing entities, or predict/add new values to the appropriate entities of this Address JSON. Expand common abbreviations (like st→street, ave→avenue), correct obvious errors (like leading zeros in numbers), generate new values to the appropriate entities, and standardize capitalization. Keep empty fields as empty strings. Do not return anything other than corrected Address JSON
Address JSON: {
  "AddNum_Pre": "",
  "Add_Number": "386",
  "AddNum_Suf": "",
  "St_PreDir": "",
  "St_Name": "Barnabye",
  "St_PosTyp": "Strret",
  "St_PosDir": "Southeest",
  "Building": "",
  "Floor": "",
  "Unit": "",
  "Room": "",
  "Uninc_Comm": "Sprngfeild Hights",
  "Inc_Muni": "Sprinfield",
  "County": "Hampdin Cnty",
  "State": "MA",
  "Zip_Code": "01190"
}
'''
    }
]

# Apply chat template
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True  # important for inference
)

# Tokenize and move to device
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Stream response
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Generate
_ = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)


System: {
  "AddNum_Pre": "",
  "Add_Number": "38",
  "AddNum_Suf": "",
  "St_PreDir": "",
  "St_Name": "barnabye",
  "St_PosTyp": "street",
  "St_PosDir": "southest",
  "Building": "",
  "Floor": "",
  "Unit": "",
  "Room": "",
  "Uninc_Comm": "springfield heights",
  "Inc_Muni": "springfield",
  "County": "hampden",
  "State": "ma",
  "Zip_Code": "1109"
}
